In [4]:
import numpy as np
from scipy.stats import distributions
from __future__ import division
normal = distributions.norm.pdf
import math
font = {'size'   : 20}
import matplotlib
matplotlib.rc('font', **font)

In [5]:
def getRows(dataBase,startInd,endInd):
    import sqlite3
    print(type(dataBase))
    input1 = dataBase+'.rdb'
    conn = sqlite3.connect(input1)
    c = conn.cursor()
    input2 = 'SELECT * FROM '+dataBase+'_ob'
    aa = c.execute(input2)
    rows = []
    count = 0
    for row in aa:
        if (count >= startInd) & (count < endInd):
            rows.append(row)
        count += 1
        if count == endInd:
            break
    return rows

In [6]:
def getStartTime(row):
    import datetime as dt
    start = dt.datetime(int(row[5][0:4]), int(row[5][5:7]), int(row[5][8:10]), int(row[5][11:13]), int(row[5][14:16]), int(float(row[5][17:-1])))
    return start

In [7]:
def getTimes(rows,startTime):
    import datetime as dt
    timeSec = [0]
    for afterInd in range(1,len(rows)):
        current = dt.datetime(int(rows[afterInd][5][0:4]), int(rows[afterInd][5][5:7]), int(rows[afterInd][5][8:10]), int(rows[afterInd][5][11:13]), int(rows[afterInd][5][14:16]), int(float(rows[afterInd][5][17:-1])))
        currentDiff = current - startTime
        currentDiffSeconds = currentDiff.days*(24*3600) + currentDiff.seconds
        timeSec.append(currentDiffSeconds)
    return timeSec

In [8]:
def extractTickData(rows,timeSec):
    import numpy as np
    buyPrices = []
    sellPrices = []
    buyTimes = []
    sellTimes = []
    buyQuantity = []
    buyLevels = []
    sellQuantity = []
    sellLevels = []
    for ii in range(len(rows)):
        if rows[ii][1] == 'buy':
            buyPrices.append(rows[ii][2])
            buyTimes.append(timeSec[ii])
            buyQuantity.append(rows[ii][3])
            buyLevels.append(rows[ii][4])
        else:
            sellPrices.append(rows[ii][2])
            sellTimes.append(timeSec[ii])
            sellQuantity.append(rows[ii][3])
            sellLevels.append(rows[ii][4])
    buyPrices = np.array(buyPrices)
    sellPrices = np.array(sellPrices)
    buyTimes = np.array(buyTimes)
    sellTimes = np.array(sellTimes)
    buyQuantity = np.array(buyQuantity)
    buyLevels = np.array(buyLevels)
    sellQuantity = np.array(sellQuantity)
    sellLevels = np.array(sellLevels)
    
    return buyPrices, sellPrices, buyTimes, sellTimes, buyQuantity, buyLevels, sellQuantity, sellLevels

In [9]:
def DHMS(seconds):
    import numpy as np
    days = np.floor(seconds/3600)
    hours = np.floor((seconds/3600) - days*24)
    minutes = np.floor(((seconds/3600) - days*24 - hours)*60)
    seconds = np.floor((((seconds/3600) - days*24 - hours)*60 - minutes)*60)
    return np.array([days,hours,minutes,seconds])

In [10]:
def timeFilterData(data,times,cutoff1,cutoff2):
    filteredData = data[(cutoff1 < times) & (times < cutoff2)]
    return filteredData

In [9]:
rows1 = getRows('huobi',20*10**6,27*10**6)

<type 'str'>


In [224]:
len(rows1)

3132800

In [11]:
rows1 = getRows('okcoin',0,4*10**6)

<type 'str'>


In [12]:
print(getStartTime(rows1[0]))

2014-01-01 21:59:52


In [13]:
startTime = getStartTime(rows1[0])
times1 = getTimes(rows1,startTime)
buyPrices1, sellPrices1, buyTimes1, sellTimes1, buyQuantity1, buyLevels1, sellQuantity1, sellLevels1 = extractTickData(rows1,times1)

In [14]:
import datetime as dt
startCutoff = dt.datetime(2014,1,2,10,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,2,23,0,0) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,4,2,0,0) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [15]:
buyPrices1_train = timeFilterData(buyPrices1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellPrices1_train = timeFilterData(sellPrices1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyTimes1_train = timeFilterData(buyTimes1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellTimes1_train = timeFilterData(sellTimes1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyQuantity1_train = timeFilterData(buyQuantity1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
buyLevels1_train = timeFilterData(buyLevels1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellQuantity1_train = timeFilterData(sellQuantity1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
sellLevels1_train = timeFilterData(sellLevels1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)

buyPrices1_trade = timeFilterData(buyPrices1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellPrices1_trade = timeFilterData(sellPrices1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyTimes1_trade = timeFilterData(buyTimes1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellTimes1_trade = timeFilterData(sellTimes1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyQuantity1_trade = timeFilterData(buyQuantity1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
buyLevels1_trade = timeFilterData(buyLevels1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellQuantity1_trade = timeFilterData(sellQuantity1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
sellLevels1_trade = timeFilterData(sellLevels1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)

In [16]:
import matplotlib.pyplot as plt
plt.plot(buyTimes1_trade,buyQuantity1_trade,'b-')
plt.xlabel('OK Coin sell times')
plt.ylabel('Ok Coin sell prices')
plt.title('Best Fit Parameters')
plt.show()

In [17]:
buyPrices1_train = buyPrices1_train[buyLevels1_train == 1]
sellPrices1_train = sellPrices1_train[sellLevels1_train == 1]
buyTimes1_train = buyTimes1_train[buyLevels1_train == 1]
sellTimes1_train = sellTimes1_train[sellLevels1_train == 1]

buyPrices1_trade = buyPrices1_trade[buyLevels1_trade == 1]
sellPrices1_trade = sellPrices1_trade[sellLevels1_trade == 1]
buyTimes1_trade = buyTimes1_trade[buyLevels1_trade == 1]
sellTimes1_trade = sellTimes1_trade[sellLevels1_trade == 1]
sellQuantity1_trade = sellQuantity1_trade[sellLevels1_trade == 1]
buyQuantity1_trade = buyQuantity1_trade[buyLevels1_trade == 1]

buyPrices1_ref = buyPrices1[buyLevels1 == 1]
sellPrices1_ref = sellPrices1[sellLevels1 == 1]
buyTimes1_ref = buyTimes1[buyLevels1 == 1]
sellTimes1_ref = sellTimes1[sellLevels1 == 1]

In [272]:
plt.plot(buyPrices1_trade)
plt.show()

In [20]:
def shallowCopy(array):              # Create shallow copies for arrays to prevent different variables from pointing to the same
    if len(array.shape) > 1:         # memory location
        newArray = np.zeros((array.shape[0],array.shape[1]))
        for ii in range(0,array.shape[0]):
            for jj in range(0,array.shape[1]):
                newArray[ii,jj] = array[ii,jj]
    else:
        newArray = np.zeros(len(array))
        for ii in range(0,len(array)):
            newArray[ii] = array[ii]
    return newArray

In [21]:
def kmeansPlus(data,K):
    N = data.shape[1]
    D = data.shape[0]
    responsibilities = np.zeros((K,N))
    meansInit = np.zeros((D,K))
    u = np.random.uniform(0,1)
    randInd = np.floor(u*N)
    meansInit[:,0] = data[:,randInd]
    minDist = np.zeros(N)
    for kk in range(1,K):
        for ii in range(0,N):
            minDist[ii] = np.sqrt(sum((data[:,ii]-meansInit[:,0])**2))
            for jj in range(0,kk-1):
                currentDist = np.sqrt(sum((data[:,ii]-meansInit[:,jj])**2))
                if currentDist < minDist[ii]:
                    minDist[ii] = currentDist
        wDist = (minDist**2)/sum(minDist**2)
        resultArray = np.random.multinomial(1, list(wDist), size=1)
        for dd in range(0,N):
            if resultArray[0][dd] == 1:
                chosenInd = dd
        meansInit[:,kk] = data[:,chosenInd]
    for ii in range(0,N):
        minAssignment = 0
        minDistance = np.sqrt(sum((data[:,ii]-meansInit[:,0])**2))
        for jj in range(1,K):
            cDistance = np.sqrt(sum((data[:,ii]-meansInit[:,jj])**2))
            if cDistance < minDistance:
                minDistance = cDistance
                minAssignment = jj
        responsibilities[minAssignment,ii] = 1
            
    oldResp = np.zeros((K,N))
    objectiveStream = []
    while sum(abs(responsibilities.flatten() - oldResp.flatten())) > 0:
        meansK = np.zeros((D,K))
        nK = np.zeros(K)
        for ii in range(0,K):
            nK[ii] = sum(responsibilities[ii,:])
            for jj in range(0,N):
                meansK[:,ii] = meansK[:,ii] + responsibilities[ii,jj]*data[:,jj]
            if nK[ii] > 0:
                meansK[:,ii] = meansK[:,ii]/nK[ii]
            else:
                u = np.random.uniform(0,1)
                pickInd = int(np.floor(u*N))
                meansK[:,ii] = data[:,pickInd]
        oldResp = shallowCopy(responsibilities)
        responsibilities = np.zeros((K,N))
        respVec = np.zeros(N)
        objectiveValue = 0
        objectiveVec = np.zeros(N)
        for ii in range(0,N):
            minAssignment = 0
            minDistance = np.sqrt(sum((data[:,ii]-meansK[:,0])**2))
            for jj in range(1,K):
                cDistance = np.sqrt(sum((data[:,ii]-meansK[:,jj])**2))
                if cDistance < minDistance:
                    minDistance = cDistance
                    minAssignment = jj
            responsibilities[minAssignment,ii] = 1
            respVec[ii] = minAssignment
            objectiveValue = objectiveValue + minDistance
            objectiveVec[ii] = minDistance
        objectiveStream.append(objectiveValue)
        #print("Objective Value is: %f" % objectiveValue)
    return responsibilities, respVec, meansK, objectiveValue, objectiveVec, np.array(objectiveStream)

In [22]:
def getViterbiPath(prices,piVec,transitionMatrix,meanMatrix,varMatrix):
    nStates = transitionMatrix.shape[0]
    nTime = len(prices[0])
    deltaMat = np.zeros((nStates,nTime))
    psiMat = np.zeros((nStates,nTime-1))
    viterbiPath = np.zeros(nTime)
    for ii in range(0,nTime):
        if ii == 0:
            for jj in range(0,nStates):
                if piVec[0][jj] > 0:
                    if varMatrix[0][jj] > 0:
                        deltaMat[jj,ii] = np.log(piVec[0][jj]) - (1/2)*np.log(2*np.pi*varMatrix[0][jj]) - (.5*(meanMatrix[0][jj] - prices[0][ii])**2)/varMatrix[0][jj] 
                    else:
                        if meanMatrix[0][jj] == prices[0][ii]:
                            deltaMat[jj,ii] = np.log(piVec[0][jj]) 
                        else:
                            deltaMat[jj,ii] = -np.Inf
                else:
                    deltaMat[jj,ii] = -np.Inf
        else:
            for jj in range(0,nStates):
                substituteCol = np.copy(transitionMatrix[:,jj])
                for kk in range(0,nStates):
                    if transitionMatrix[kk,jj] == 0:
                        substituteCol[kk] = -np.Inf
                    else:
                        substituteCol[kk] = np.log(transitionMatrix[kk,jj])
                if varMatrix[0,jj] > 0:
                    deltaMat[jj,ii] = max(deltaMat[:,ii-1] + substituteCol) - (1/2)*np.log(2*np.pi*varMatrix[0,jj]) - (.5*(prices[0][ii]-meanMatrix[0,jj])**2)/varMatrix[0,jj]
                else:
                    if prices[0][ii] == meanMatrix[0,jj]:
                        deltaMat[jj,ii] = max(deltaMat[:,ii-1] + substituteCol) 
                    else:
                        deltaMat[jj,ii] = -np.Inf
                psiMat[jj,ii-1] = np.argmax(deltaMat[:,ii-1] + substituteCol)
        if all(deltaMat[:,ii] == -np.inf):
            print("All -inf")
    viterbiPath[-1] = np.argmax(deltaMat[:,nTime-1])
    for ii in range(2,nTime+1):
        viterbiPath[-ii] = psiMat[viterbiPath[-ii+1],nTime-ii]
    return viterbiPath

In [23]:
def getLogLik(prices,stateVec,piVec,transitionMatrix,meanMatrix,varMatrix):
    logLik = 0
    for ii in range(0,len(prices[0])):
        if varMatrix[0,stateVec[0][ii]] > 0:
            logLik = logLik - ((.5*(prices[0][ii]-meanMatrix[0,stateVec[0][ii]])**2)/(varMatrix[0,stateVec[0][ii]])) - (1/2)*np.log(2*np.pi*varMatrix[0,stateVec[0][ii]])
        else:
            if prices[0][ii] != meanMatrix[0,stateVec[0][ii]]:
                logLik = -np.Inf
                break
        if ii == 0:
            logLik = logLik + np.log(piVec[0,stateVec[0][ii]])
        if ii < len(prices)-1:
            logLik = logLik + np.log(transitionMatrix[stateVec[0][ii],stateVec[0][ii+1]])
    return logLik

In [24]:
def trainDataHMM(prices, nStates, nComponents, logLikThresh):
    nTime = len(prices[0])
    respMat, stateVec, meansK, objValue, objVec, ojbStream = kmeansPlus(prices,nStates)
    stateVec = np.reshape(stateVec,(1,len(stateVec)))
    meanMatrix = np.zeros((nComponents,nStates))
    varMatrix = np.zeros((nComponents,nStates))
    for ii in range(0,nStates):
        nGroup = sum(stateVec[0] == ii)
        if nGroup == 0:
            print('initial empty cluster',ii)
        dataCurrent = prices[0,stateVec[0] == ii]
        if nComponents > 1:
            respMatS, stateVecS, meansKS, objValueS, objVecS, ojbStreamS = kmeansPlus(dataCurrent,nComponents)
            for jj in range(0,nComponents):
                meanMatrix[ii,jj] = np.mean(dataCurrent[stateVecS[0] == jj])
                varMatrix[ii,jj] = np.var(dataCurrent[stateVecS[0] == jj])
        else:
            meanMatrix[0,ii] = np.mean(dataCurrent)
            varMatrix[0,ii] = np.var(dataCurrent)
            
    transitionMatrix = np.zeros((nStates,nStates))
    for ii in range(0,nStates):
        for jj in range(0,nStates):
            den = 0
            for kk in range(0,nTime-1):
                if (stateVec[0,kk] == ii) & (stateVec[0,kk+1] == jj):
                    transitionMatrix[ii,jj] = transitionMatrix[ii,jj] + 1
                if stateVec[0,kk] == ii:
                    den = den + 1
            if den > 0:
                transitionMatrix[ii,jj] = transitionMatrix[ii,jj]/den
    
    initialObs = prices[0][0]
    invDist = 1/abs(meansK-initialObs)
    piVec = invDist/sum(invDist[0])

    currentLogLik = getLogLik(prices,stateVec,piVec,transitionMatrix,meanMatrix,varMatrix)
    #print("Initial Log Lik is: ", currentLogLik)
    #print("Initial pi is: ", piVec)
    #print("Initial transition is: ", transitionMatrix)
    #print("Initial mean is: ", meanMatrix)
    #print("Initial var is: ", varMatrix)
    #print("Initial path is: ", stateVec)
    likVec = [currentLogLik]
    
    newLogLik = 10

    while abs(newLogLik - currentLogLik) > logLikThresh:
        viterbiPath = getViterbiPath(prices,piVec,transitionMatrix, meanMatrix,varMatrix)
        viterbiPath = np.reshape(viterbiPath,(1,len(viterbiPath)))
        intermediateLogLik = getLogLik(prices,viterbiPath,piVec,transitionMatrix,meanMatrix,varMatrix)
        #print("Intermediate log lik is: ",intermediateLogLik)
        
        meanMatrixNew = np.zeros((nComponents,nStates))
        varMatrixNew = np.zeros((nComponents,nStates))
        for ii in range(0,nStates):
            nGroup = sum(viterbiPath[0] == ii)
            if nGroup == 0:
                print('empty cluster',ii)
            dataCurrent = prices[0,viterbiPath[0] == ii]
            if nComponents > 1:
                respMatS, stateVec, meansKS, objValueS, objVecS, ojbStreamS = kmeansPlus(dataCurrent,nComponents)
                for jj in range(0,nComponents):
                    meanMatrixNew[ii,jj] = np.mean(dataCurrent[stateVec[0] == jj])
                    varMatrixNew[ii,jj] = np.var(dataCurrent[stateVec[0] == jj])   
            else:
                meanMatrixNew[0,ii] = np.mean(dataCurrent)
                varMatrixNew[0,ii] = np.var(dataCurrent)
        transitionMatrixNew = np.zeros((nStates,nStates))
      
        for ii in range(0,nStates):
            for jj in range(0,nStates):
                den = 0
                for kk in range(0,nTime-1):
                    if (viterbiPath[0,kk] == ii) & (viterbiPath[0,kk+1] == jj):
                        transitionMatrixNew[ii,jj] = transitionMatrixNew[ii,jj] + 1
                    if viterbiPath[0,kk] == ii:
                        den = den + 1
                if den > 0:
                    transitionMatrixNew[ii,jj] = transitionMatrixNew[ii,jj]/den
        meansKNew = np.zeros((nComponents,nStates))
        for ii in range(0,nStates):
            meansKNew[0,ii] = np.mean(prices[0][viterbiPath[0] == ii])
        invDistNew = 1/abs(meansKNew-initialObs)
        piVecNew = invDistNew/sum(invDistNew[0])
        
        newLogLik = getLogLik(prices,viterbiPath,piVecNew,transitionMatrixNew,meanMatrixNew,varMatrixNew)
        
        if newLogLik > currentLogLik:
            likVec.append(newLogLik)
            piVec = np.copy(piVecNew)
            transitionMatrix = np.copy(transitionMatrixNew)
            meanMatrix = np.copy(meanMatrixNew)
            varMatrix = np.copy(varMatrixNew)
            stateVec = np.copy(viterbiPath)
            currentLogLik = newLogLik
            
            #print("New Log Lik is: ", currentLogLik)
            #print("New pi is: ", piVec)
            #print("New transition is: ", transitionMatrix)
            #print("New mean is: ", meanMatrix)
            #print("New var is: ", varMatrix)
            #print("New path is: ", stateVec)
        else:
            #print("Worse Candidate Log Lik is: ", newLogLik)
            #print("Worse path is: ",viterbiPath)
            break
        
    return piVec, transitionMatrix, meanMatrix, varMatrix, likVec, stateVec

In [24]:
trainBuy = np.reshape(buyPrices1_train,(1,buyPrices1_train.shape[0]))
piVec, transitionMatrix, meanMatrix, varMatrix, likVec, stateVec = trainDataHMM(trainBuy, 2, 1, 0.1)

In [25]:
yTrain = trainBuy[0]
yTrade = buyPrices1_trade
timeIndTrain = np.linspace(0,len(yTrain)-1,len(yTrain))
timeIndTrade = len(yTrain) + np.linspace(0,len(yTrade)-1,len(yTrade))
plt.plot(timeIndTrain[stateVec[0]==0],yTrain[stateVec[0]==0],'b.',label='Train Clust 1')
plt.plot(timeIndTrain[stateVec[0]==1],yTrain[stateVec[0]==1],'g.',label='Train Clust 2')
plt.plot(timeIndTrade,yTrade,'r.',label='Trade')
plt.legend(loc=(0,.7))
plt.ylabel('BTC Price')
plt.xlabel('Tick')
plt.title('Training and Trade')
plt.show()

In [25]:
def balanceLogSum(logs):
    if sum(logs != -np.Inf) == 0:
        return -np.Inf
    else:
        logM = logs[logs != -np.Inf]
        maxLS = max(logM)
        lb = maxLS - 50
        return lb + np.log(sum(np.exp(logM - lb)))

In [26]:
def getAlpha(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix,stopFlag):
       
    alphaMat = np.zeros((nStates,len(currentWindow)))
    logAlphaNorm = np.zeros((nStates,len(currentWindow)))
    
    for iState in range(0,nStates):
        var = varMatrix[0][iState]
        if var > 0:
            if piVec[0][iState] > 0:
                alphaMat[iState,0] = np.log(piVec[0][iState]) - ((.5*(currentWindow[0]-meanMatrix[0][iState])**2)/var)-(1/2)*np.log(2*np.pi*var)
            else:
                alphaMat[iState,0] = -np.Inf
        else:
            if currentWindow[0] == meanMatrix[0][iState]:
                if piVec[0][iState] > 0:
                    alphaMat[iState,0] = np.log(piVec[0][iState])
                else:
                    alphaMat[iState,0] = -np.Inf
            else:
                alphaMat[iState,0] = -np.Inf
        if all(alphaMat[:,0] == -np.Inf):
            print("both -inf")
            print("alpha: ",alphaMat)
            print("pi: ",piVec[0])
            print("var: ",varMatrix[0])
            print("mean: ",meanMatrix[0])
            print("value: ",currentWindow[0])
            print("transition: ",transitionMatrix)
            print("window: ",currentWindow)
            return
        
    for iTime in range(1,len(currentWindow)):
        for iState in range(0,nStates):
            logTransitionColumn = np.log(transitionMatrix[:,iState])
            logSum = logTransitionColumn + alphaMat[:,iTime-1]
            var = varMatrix[0][iState]
            if var > 0:
                logObs = - ((.5*(currentWindow[iTime]-meanMatrix[0][iState])**2)/var)-(1/2)*np.log(2*np.pi*var)
            else:
                if currentWindow[iTime] == meanMatrix[0][iState]:
                    logObs = 0
                else:
                    logObs = -np.Inf
            alphaMat[iState,iTime] = logObs + balanceLogSum(logSum)
            if math.isnan(alphaMat[iState,iTime]):
                print("alpha: ",alphaMat[:,0:iTime+1])
                print("time: ",iTime)
                print("state: ",iState)
                print("log t: ",logTransitionColumn)
                print("trans mat: ",transitionMatrix)
                print("log sum: ",logSum)
                print("var vec: ",varMatrix)
                print("logObs: ",logObs)
                print("balSum: ",balanceLogSum(logSum))
                return
    for ii in range(0,len(currentWindow)):
        balancedLogSum = balanceLogSum(alphaMat[:,ii])
        if balancedLogSum == -np.Inf:
            print('error alpha')
            print('alpha column: ',alphaMat[:,ii])
            print('alpha: ',alphaMat)
            print("trans mat: ",transitionMatrix)
            print("log sum: ",logSum)
            print("var vec: ",varMatrix)
            print("logObs: ",logObs)
            return
        for iS in range(0,nStates):
            if alphaMat[iS,ii] == -np.Inf:
                logAlphaNorm[iS,ii] = -np.Inf
            else:
                logAlphaNorm[iS,ii] = alphaMat[iS,ii] - balancedLogSum
    alphaNorm = np.exp(logAlphaNorm)
    logPass = alphaMat[:,len(currentWindow)-1]
    logPObs = balanceLogSum(logPass)
    
    return alphaMat, alphaNorm, logPObs

In [27]:
def getBeta(currentWindow,nStates,transitionMatrix,meanMatrix,varMatrix):
    betaMat = np.zeros((nStates,len(currentWindow)))
    logBetaNorm = np.zeros((nStates,len(currentWindow)))
    #if any(varMatrix[0] == 0):
        #print("zero var")
    for iTime in range(2,len(currentWindow)+1):
        logObs = np.zeros(nStates)
        for iS in range(0,nStates):
            var = varMatrix[0][iS]
            if var > 0:
                logObs[iS] = -((.5*(currentWindow[-iTime+1]-meanMatrix[0][iS])**2)/var)-(1/2)*np.log(2*np.pi*var)
            else:
                if currentWindow[-iTime + 1] == meanMatrix[0][iS]:
                    logObs[iS] = 0
                else:
                    logObs[iS] = -np.Inf
        for iState in range(0,nStates):
            substituteRow = np.copy(transitionMatrix[iState,:])
            for jj in range(0,nStates):
                if transitionMatrix[iState,jj] == 0:
                    substituteRow[jj] = -np.Inf
                else:
                    substituteRow[jj] = np.log(transitionMatrix[iState,jj])
            logSum = substituteRow + betaMat[:,-iTime+1] + logObs
            betaMat[iState,-iTime] = balanceLogSum(logSum)
    for ii in range(0,len(currentWindow)):
        logSum = betaMat[:,ii]
        balancedLogSum = balanceLogSum(logSum)
        logBetaNorm[:,ii] = betaMat[:,ii] - balancedLogSum
    betaNorm = np.exp(logBetaNorm)
    return betaMat, betaNorm

In [28]:
def tradeHMM(buyData, tradeStart, nStates, nComponents, piVec, transitionMatrix, meanMatrix, varMatrix, npred, windowLength, threshold, discount):
    #npred = len(buyData[0]) - tradeStart -1
    predictionDir = np.zeros(npred)
    predictionDirMomentum = np.zeros(npred)
    truthDir = np.zeros(npred)
    prediction = np.zeros(npred)
    changeState = np.zeros(npred)
    truth = np.zeros(npred)
    betPrice = np.zeros(npred)
    stateTrack = [[] for ii in range(0,npred-1)]
    meanTrack = [[] for ii in range(0,npred-1)]
    meanAll = np.zeros((nStates,npred))
    varAll = np.zeros((nStates,npred))
    pTrack = []
    piVec0 = np.copy(piVec)
    transitionMatrix0 = np.copy(transitionMatrix)
    meanMatrix0 = np.copy(meanMatrix)
    varMatrix0 = np.copy(varMatrix)
    """
    piVec = np.copy(piVec + np.random.uniform(0,1,2))
    piVec = np.copy(piVec / sum(piVec[0]))
    transitionMatrix = np.copy(transitionMatrix.flatten() + np.random.uniform(0,1,4))
    transitionMatrix = np.reshape(transitionMatrix,(2,2))
    transitionMatrix[0,:] = transitionMatrix[0,:]/sum(transitionMatrix[0,:])
    transitionMatrix[1,:] = transitionMatrix[1,:]/sum(transitionMatrix[1,:])
    meanMatrix = np.copy(meanMatrix + np.random.uniform(-1,1,2))
    varMatrix = np.copy(varMatrix + np.random.uniform(-min(varMatrix.flatten())/2,min(varMatrix.flatten())/2,2))
    """
    for uu in range(tradeStart,tradeStart+npred):
        pObsNew = 10
        pObsCurrent = 0
        currentWindow = buyData[0,uu-windowLength:uu]
        pTracker = []
        if uu > tradeStart:
            piVec = np.copy(piVec0)
            transitionMatrix = np.copy(transitionMatrix0)
            meanMatrix = np.copy(meanMatrix0)
            varMatrix = np.copy(varMatrix0)
        while pObsNew - pObsCurrent > threshold:
                   
            alphaCurrent, alphaNorm, pObsCurrent = getAlpha(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix,0)
            betaCurrent, betaNorm = getBeta(currentWindow,nStates,transitionMatrix,meanMatrix,varMatrix)
            pTracker.append(pObsCurrent)
            
            gammaCurrent = np.zeros((nStates,windowLength))
            for hh in range(0,windowLength):
                logSum = np.copy(alphaCurrent[:,hh]+betaCurrent[:,hh])
                gammaCurrent[:,hh] = np.copy(alphaCurrent[:,hh] + betaCurrent[:,hh] - balanceLogSum(logSum))
                
            gammaCurrent = np.exp(gammaCurrent)
            newPiVec = np.reshape(gammaCurrent[:,0],(nComponents,nStates))
            newTransitionMatrix = np.zeros((nStates,nStates))
            originalGamma = np.zeros(nStates)
            if discount == 1:
                discountWindow0 = np.ones(windowLength-1)
            else:
                discountWindow0 = discount*((1-discount)**(windowLength-1-(np.linspace(0,windowLength-2,windowLength-1))))
            for ii in range(0,nStates):
                originalGamma[ii] = sum(discountWindow0*gammaCurrent[ii,0:windowLength-1])
                
            compareGamma = np.zeros(nStates)
            currentDens = np.zeros(windowLength-1)
            for cc in range(0,windowLength-1):
                currentDen = 0
                for dd in range(0,nStates):
                    for ee in range(0,nStates):
                        if varMatrix[0][ee] == 0:
                            if currentWindow[cc+1] == meanMatrix[0][ee]:
                                currentDen = currentDen + alphaNorm[dd,cc]*transitionMatrix[dd,ee]*betaNorm[ee,cc+1]
                        else:
                            currentDen = currentDen + alphaNorm[dd,cc]*transitionMatrix[dd,ee]*normal(currentWindow[cc+1],meanMatrix[0][ee],np.sqrt(varMatrix[0][ee]))*betaNorm[ee,cc+1]
                currentDens[cc] = currentDen           
            for ii in range(0,nStates):
                for jj in range(0,nStates):
                    for cc in range(0,windowLength-1):
                        currentDen = currentDens[cc]
                        if currentDen > 0:
                            if varMatrix[0][jj] == 0:
                                if currentWindow[cc+1] == meanMatrix[0][jj]:
                                    if discount == 1:
                                        newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] + (alphaNorm[ii,cc]*transitionMatrix[ii,jj]*betaNorm[jj,cc+1])/currentDen
                                    else:
                                        newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] + discount*((1-discount)**(windowLength-1-cc))*(alphaNorm[ii,cc]*transitionMatrix[ii,jj]*betaNorm[jj,cc+1])/currentDen      
                            else:
                                if discount == 1:
                                    newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] + (alphaNorm[ii,cc]*transitionMatrix[ii,jj]*normal(currentWindow[cc+1],meanMatrix[0][jj],np.sqrt(varMatrix[0][jj]))*betaNorm[jj,cc+1])/currentDen
                                else:
                                    newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] + discount*((1-discount)**(windowLength-1-cc))*(alphaNorm[ii,cc]*transitionMatrix[ii,jj]*normal(currentWindow[cc+1],meanMatrix[0][jj],np.sqrt(varMatrix[0][jj]))*betaNorm[jj,cc+1])/currentDen
                            if jj == 0:
                                for ff in range(0,nStates):
                                    if varMatrix[0][ff] > 0:
                                        if discount == 1:
                                            compareGamma[ii] = compareGamma[ii] + (alphaNorm[ii,cc]*transitionMatrix[ii,ff]*normal(currentWindow[cc+1],meanMatrix[0][ff],np.sqrt(varMatrix[0][ff]))*betaNorm[ff,cc+1])/currentDen
                                        else:
                                            compareGamma[ii] = compareGamma[ii] + discount*((1-discount)**(windowLength-1-cc))*(alphaNorm[ii,cc]*transitionMatrix[ii,ff]*normal(currentWindow[cc+1],meanMatrix[0][ff],np.sqrt(varMatrix[0][ff]))*betaNorm[ff,cc+1])/currentDen    
                                    else:
                                        if currentWindow[cc+1] == meanMatrix[0][ff]:
                                            if discount == 1:
                                                compareGamma[ii] = compareGamma[ii] + (alphaNorm[ii,cc]*transitionMatrix[ii,ff]*betaNorm[ff,cc+1])/currentDen
                                            else:
                                                compareGamma[ii] = compareGamma[ii] + discount*((1-discount)**(windowLength-1-cc))*(alphaNorm[ii,cc]*transitionMatrix[ii,ff]*betaNorm[ff,cc+1])/currentDen
                    if originalGamma[ii] > 0:
                        newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] / originalGamma[ii]
                    else:
                        if newTransitionMatrix[ii,jj] != 0:
                            print('error trans compute: original')
                                       
            newMeanMatrix = np.zeros((nComponents,nStates))
            newVarMatrix = np.zeros((nComponents,nStates))
            if discount == 1:
                discountWindow = np.ones(windowLength)
            else:
                discountWindow = (discount*((1-discount)**(windowLength-1-(np.linspace(0,windowLength-1,windowLength)))))
            for ii in range(0,nStates):
                if sum(gammaCurrent[ii,:]) > 0:
                    valStore = np.copy(gammaCurrent[ii,:]*currentWindow)
                    newMeanMatrix[0,ii] = sum(discountWindow*valStore)/sum(discountWindow*gammaCurrent[ii,:])
                    valStore2 = np.copy(gammaCurrent[ii,:]*((currentWindow-newMeanMatrix[0,ii])**2))
                    newVarMatrix[0,ii] = sum(discountWindow*valStore2)/sum(discountWindow*gammaCurrent[ii,:])
            """
            if any(newVarMatrix[0] == 0):
                print("gamma: ",gammaCurrent)
                print("old pi: ",piVec)
                print("means: ",newMeanMatrix)
                print("vars: ",newVarMatrix)
                print("window: ",currentWindow)
                print("trans: ",newTransitionMatrix)
                print("alpha: ",alphaCurrent)
                print("beta: ",betaCurrent)
                print("alpha norm: ",alphaNorm)
                print("beta norm: ",betaNorm)
                print("original gamma: ",originalGamma)
                print("compare gamma: ",compareGamma)
            """
            transitionNumbers = newTransitionMatrix.flatten()
            for iTrans in range(0,len(transitionNumbers)):
                if math.isnan(transitionNumbers[iTrans]):
                    print("Nan transition:", newTransitionMatrix)
                    print("old transition: ", transitionMatrix)
                    print("gamma: ",originalGamma)
                    print("alpha norm: ",alphaNorm)
                    print("beta norm: ",betaNorm)
                    print("mean matrix: ",meanMatrix)
                    print("var matrix: ",varMatrix)
                    return
            if sum(newTransitionMatrix.flatten() != 0) == 0:
                print('trans error: all zero')
                print("old transition: ", transitionMatrix)
                print("gamma: ",originalGamma)
                print("alpha norm: ",alphaNorm)
                print("beta norm: ",betaNorm)
                print("mean matrix: ",meanMatrix)
                print("var matrix: ",varMatrix)
                return
            for ii in range(0,len(newVarMatrix[0])):
                if newVarMatrix[0][ii] == 0:
                    newVarMatrix[0][ii] = 10**-10
            newAlpha, newAlphaNorm, pObsNew = getAlpha(currentWindow,nStates,newPiVec,newTransitionMatrix,newMeanMatrix,newVarMatrix,1)
            
            pTracker.append(pObsNew)
            """    
            if pObsCurrent > pObsNew + .01:
                print("original gamma", originalGamma)
                print("compare gamma", compareGamma)
                print("old transition",transitionMatrix)
                print("new transition", newTransitionMatrix)
                print("old pi", piVec)
                print("new pi", newPiVec)
                print("old mean", meanMatrix)
                print("new mean", newMeanMatrix)
                print("old var", varMatrix)
                print("new var", newVarMatrix)
                print("old alpha",alphaCurrent)
                print("new alpha",newAlpha)
                print("p old",pObsCurrent)
                print("p new",pObsNew)
            
            if uu == tradeStart:
                print("original gamma", originalGamma)
                print("compare gamma", compareGamma)
                print("old transition",transitionMatrix)
                print("new transition", newTransitionMatrix)
                print("old pi", piVec)
                print("new pi", newPiVec)
                print("old mean", meanMatrix)
                print("new mean", newMeanMatrix)
                print("old var", varMatrix)
                print("new var", newVarMatrix)
                print("old alpha",alphaCurrent)
                print("new alpha",newAlpha)
                print("p old",pObsCurrent)
                print("p new",pObsNew)
                return
            """
            if pObsNew > pObsCurrent:
                piVec = np.copy(newPiVec)
                transitionMatrix = np.copy(newTransitionMatrix)
                meanMatrix = np.copy(newMeanMatrix)
                varMatrix = np.copy(newVarMatrix)
            else:
                break
        
        viterbiPath = getViterbiPath(np.reshape(currentWindow,(1,len(currentWindow))),piVec,transitionMatrix, meanMatrix,varMatrix)
        maxNextState = np.argmax(transitionMatrix[viterbiPath[-1],:])
        predictionDir[uu-tradeStart] = np.sign(meanMatrix[0,maxNextState]-buyData[0,uu-1])
        if viterbiPath[-1] != np.argmax(transitionMatrix[viterbiPath[-1],:]):
            changeState[uu-tradeStart] = 1
        else:
            changeState[uu-tradeStart] = 0
        if (currentWindow[-1]-meanMatrix[0,viterbiPath[-1]])/np.sqrt(varMatrix[0,viterbiPath[-1]]) > .7:
            predictionDirMomentum[uu-tradeStart] = -1
        elif (currentWindow[-1]-meanMatrix[0,viterbiPath[-1]])/np.sqrt(varMatrix[0,viterbiPath[-1]]) < -.7:
            predictionDirMomentum[uu-tradeStart] = 1
        else:
            predictionDirMomentum[uu-tradeStart] = 0
            
        truthDir[uu-tradeStart] = np.sign(buyData[0,uu]-buyData[0,uu-1])
        prediction[uu-tradeStart] = meanMatrix[0,np.argmax(transitionMatrix[viterbiPath[-1],:])]
        for iS in range(0,nStates):
            meanAll[iS,uu-tradeStart] = meanMatrix[0,iS]
            varAll[iS,uu-tradeStart] = varMatrix[0,iS]
        if uu > tradeStart:
            stateTrack[uu-tradeStart-1].append(viterbiPath[-1])
            meanTrack[uu-tradeStart-1].append(meanMatrix[0,viterbiPath[-1]])
            backStep = min(windowLength-1,uu-tradeStart-1)
            if backStep > 0:
                for jj in range(1,backStep+1):
                    stateTrack[uu-tradeStart-1-jj].append(viterbiPath[-1-jj])
                    meanTrack[uu-tradeStart-1-jj].append(meanMatrix[0,viterbiPath[-1-jj]])
        truth[uu-tradeStart] = buyData[0,uu]
        betPrice[uu-tradeStart] = buyData[0,uu-1]
        pTrack.append(pTracker)
        #print("p track", pTracker)
        #print("pred",meanMatrix[0,np.argmax(transitionMatrix[viterbiPath[-1],:])])
        #print("actual",buyData[0,uu+1])
        #print("pred dir",prediction[uu])
        #print("actual dir",truth[uu])
        #print("next pi",piVec)
        #print("next transition",transitionMatrix)
        #print("next mean",meanMatrix)
        #print("next var",varMatrix)
           
    return truthDir, predictionDir, predictionDirMomentum, betPrice, stateTrack, meanTrack, meanAll, varAll, changeState, truth, prediction, pTrack

In [29]:
def FscoreTradeParam(predictionDirList, truthDir, windowLength):
    FscoreDir = np.zeros(len(predictionDirList[0]))
    nStrategies = len(predictionDirList)
    for ii in range(0,len(predictionDirList[0])):
        if ii == 0:
            runsum = 0
            for jj in range(0,nStrategies):
                runsum = runsum + predictionDirList[jj][ii]
            FscoreDir[ii] = runsum/nStrategies
        else:
            backStep = min(windowLength,ii)
            Fscores = np.zeros(nStrategies)
            preds = np.zeros(nStrategies)
            for jj in range(0,nStrategies):
                histPred = predictionDirList[jj][ii-backStep:ii]
                truth = truthDir[ii-backStep:ii]
                preds[jj] = predictionDirList[jj][ii]
                releventSignals = (truth != 0)
                recall = sum(histPred[releventSignals] == truth[releventSignals])/sum(releventSignals)
                retrievedSignals = (histPred != 0)
                precision = sum(histPred[retrievedSignals] == truth[retrievedSignals])/sum(retrievedSignals)
                Fscores[jj] = 2*precision*recall/(precision+recall)
            FscoreDir[ii] = sum(Fscores*preds)/sum(Fscores)
    return FscoreDir

In [44]:
trainBuy = np.reshape(buyPrices1_train,(1,buyPrices1_train.shape[0]))
piVecB, transitionMatrixB, meanMatrixB, varMatrixB, likVecB, stateVecB = trainDataHMM(trainBuy, 4, 1, 0.1)
trainSell = np.reshape(sellPrices1_train,(1,sellPrices1_train.shape[0]))
piVecS, transitionMatrixS, meanMatrixS, varMatrixS, likVecS, stateVecS = trainDataHMM(trainSell, 4, 1, 0.1)

In [45]:
buyCombine = np.zeros(50 + len(buyPrices1_trade))
sellCombine = np.zeros(50 + len(sellPrices1_trade))
buyCombine[0:50] = np.copy(buyPrices1_train[len(buyPrices1_train)-50:len(buyPrices1_train)]) 
buyCombine[50:] = np.copy(buyPrices1_trade) 
sellCombine[0:50] = np.copy(sellPrices1_train[len(sellPrices1_train)-50:len(sellPrices1_train)]) 
sellCombine[50:] = np.copy(sellPrices1_trade) 
timesPred = np.copy(buyTimes1_trade)

In [89]:
len(buyCombine) - tradeStart

29490

In [100]:
import time
startExec = time.time()
buyData = np.reshape(buyCombine,(1,len(buyCombine)))
sellData = np.reshape(sellCombine,(1,len(sellCombine)))
tradeStart = 50
windowLength = 10
discount = 1
npred = len(buyCombine) - tradeStart
nStates = 4
truthDirB,predictionDirB,predictionDirMomentumB,betPriceB,stateTrackB,meanTrackB,meanAllB,varAllB,changeStateB,truthB,predictionB,pTrackB = tradeHMM(buyData, tradeStart, nStates, 1, piVecB, transitionMatrixB, meanMatrixB, varMatrixB, npred, windowLength, 0.1, discount)
truthDirS,predictionDirS,predictionDirMomentumS,betPriceS,stateTrackS,meanTrackS,meanAllS,varAllS,changeStateS,truthS,predictionS,pTrackS = tradeHMM(sellData, tradeStart, nStates, 1, piVecS, transitionMatrixS, meanMatrixS, varMatrixS, npred, windowLength, 0.1, discount)
endExec = time.time()
totalTime = endExec - startExec
print "total time spent: %f" % totalTime

total time spent: 3314.193000


C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:34: RuntimeWarning: divide by zero encountered in log


In [221]:
np.savetxt('predictionDirBuyMomentum_0_4.txt', predictionDirMomentumB, delimiter=',')
np.savetxt('predictionDirSellMomentum_0_4.txt', predictionDirMomentumS, delimiter=',')
np.savetxt('predictionDirBuy_0_4.txt', predictionDirB, delimiter=',')
np.savetxt('predictionDirSell_0_4.txt', predictionDirS, delimiter=',')
np.savetxt('predictionBuy_0_4.txt', predictionB, delimiter=',')
np.savetxt('predictionSell_0_4.txt', predictionS, delimiter=',')
with file('meanBuy_0_4.txt', 'w') as outfile:
    for data_slice in meanAllB:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('meanSell_0_4.txt', 'w') as outfile:
    for data_slice in meanAllS:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varBuy_0_4.txt', 'w') as outfile:
    for data_slice in varAllB:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varSell_0_4.txt', 'w') as outfile:
    for data_slice in varAllS:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')

In [101]:
print "HMM Buy accuracy: ", sum(truthDirB == predictionDirB)/len(truthDirB)
print "Momentum HMM Buy accuracy: ", sum(truthDirB == predictionDirMomentumB)/len(truthDirB)
print "HMM Sell accuracy: ", sum(truthDirS == predictionDirS)/len(truthDirS)
print "Momentum HMM Sell accuracy: ", sum(truthDirS == predictionDirMomentumS)/len(truthDirS)

HMM Buy accuracy:  0.383282468633
Momentum HMM Buy accuracy:  0.503797897592
HMM Sell accuracy:  0.393964055612
Momentum HMM Sell accuracy:  0.522550016955


In [258]:
print "RMSE Buy: ", np.sqrt(np.mean(abs(predictionB-truthB)**2))
print "RMSE Sell: ", np.sqrt(np.mean(abs(predictionS-truthS)**2))

RMSE Buy:  2.63814194816
RMSE Sell:  4.74119585083


In [271]:
plt.plot(betPriceB,'b.')
plt.plot(betPriceS,'r.')
plt.show()

In [236]:
profit, bankTracker, assetTracker = simple_trade(betPriceB,betPriceS,truthB[-1],truthS[-1],predictionDirB,predictionDirS)
print "Profit: $", profit

Profit: $ -1877.904


In [104]:
piVecB2, transitionMatrixB2, meanMatrixB2, varMatrixB2, likVecB2, stateVecB2 = trainDataHMM(trainBuy, 8, 1, 0.1)
piVecS2, transitionMatrixS2, meanMatrixS2, varMatrixS2, likVecS2, stateVecS2 = trainDataHMM(trainSell, 8, 1, 0.1)
import time
startExec = time.time()
tradeStart = 50
windowLength = 10
discount = 1
npred = len(buyCombine) - tradeStart
nStates = 8
truthDirB2,predictionDirB2,predictionDirMomentumB2,betPriceB2,stateTrackB2,meanTrackB2,meanAllB2,varAllB2,changeStateB2,truthB2,predictionB2,pTrackB2 = tradeHMM(buyData, tradeStart, nStates, 1, piVecB2, transitionMatrixB2, meanMatrixB2, varMatrixB2, npred, windowLength, 0.1, discount)
truthDirS2,predictionDirS2,predictionDirMomentumS2,betPriceS2,stateTrackS2,meanTrackS2,meanAllS2,varAllS2,changeStateS2,truthS2,predictionS2,pTrackS2 = tradeHMM(sellData, tradeStart, nStates, 1, piVecS2, transitionMatrixS2, meanMatrixS2, varMatrixS2, npred, windowLength, 0.1, discount)
endExec = time.time()
totalTime = endExec - startExec
print "total time spent: %f" % totalTime
print "HMM Buy accuracy: ", sum(truthDirB2 == predictionDirB2)/len(truthDirB2)
print "Momentum HMM Buy accuracy: ", sum(truthDirB2 == predictionDirMomentumB2)/len(truthDirB2)
print "HMM Sell accuracy: ", sum(truthDirS2 == predictionDirS2)/len(truthDirS2)
print "Momentum HMM Sell accuracy: ", sum(truthDirS2 == predictionDirMomentumS2)/len(truthDirS2)

C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:34: RuntimeWarning: divide by zero encountered in log
C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:38: RuntimeWarning: overflow encountered in double_scalars
C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:11: RuntimeWarning: overflow encountered in double_scalars
C:\Users\draj36\Anaconda2\lib\site-packages\scipy\stats\_continuous_distns.py:78: RuntimeWarning: overflow encountered in square
  return exp(-x**2/2.0) / _norm_pdf_C
C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:29: RuntimeWarning: overflow encountered in double_scalars


HMM Buy accuracy:  0.369040352662
Momentum HMM Buy accuracy:  0.505561207189
HMM Sell accuracy:  0.374499830451
Momentum HMM Sell accuracy:  0.506578501187
Profit: $ -5814.097


In [218]:
np.savetxt('predictionDirBuyMomentum_0_8.txt', predictionDirMomentumB2, delimiter=',')
np.savetxt('predictionDirSellMomentum_0_8.txt', predictionDirMomentumS2, delimiter=',')
np.savetxt('predictionDirBuy_0_8.txt', predictionDirB2, delimiter=',')
np.savetxt('predictionDirSell_0_8.txt', predictionDirS2, delimiter=',')
np.savetxt('predictionBuy_0_8.txt', predictionB2, delimiter=',')
np.savetxt('predictionSell_0_8.txt', predictionS2, delimiter=',')
with file('meanBuy_0_8.txt', 'w') as outfile:
    for data_slice in meanAllB2:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('meanSell_0_8.txt', 'w') as outfile:
    for data_slice in meanAllS2:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varBuy_0_8.txt', 'w') as outfile:
    for data_slice in varAllB2:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varSell_0_8.txt', 'w') as outfile:
    for data_slice in varAllS2:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')

In [256]:
print "RMSE Buy: ", np.sqrt(np.mean(abs(predictionB2-truthB)**2))
print "RMSE Sell: ", np.sqrt(np.mean(abs(predictionS2-truthS)**2))

RMSE Buy:  2.64865141631
RMSE Sell:  163.4796027


In [235]:
profit2, bankTracker2, assetTracker2 = simple_trade(betPriceB2,betPriceS2,truthB2[-1],truthS2[-1],predictionDirB2,predictionDirS2)
print "Profit: $", profit2

Profit: $ 3698.09300001


In [132]:
piVecB0, transitionMatrixB0, meanMatrixB0, varMatrixB0, likVecB0, stateVecB0 = trainDataHMM(trainBuy, 2, 1, 0.1)
piVecS0, transitionMatrixS0, meanMatrixS0, varMatrixS0, likVecS0, stateVecS0 = trainDataHMM(trainSell, 2, 1, 0.1)
import time
startExec = time.time()
tradeStart = 50
windowLength = 10
discount = 1
npred = len(buyCombine) - tradeStart
nStates = 2
truthDirB0,predictionDirB0,predictionDirMomentumB0,betPriceB0,stateTrackB0,meanTrackB0,meanAllB0,varAllB0,changeStateB0,truthB0,predictionB0,pTrackB0 = tradeHMM(buyData, tradeStart, nStates, 1, piVecB0, transitionMatrixB0, meanMatrixB0, varMatrixB0, npred, windowLength, 0.1, discount)
truthDirS0,predictionDirS0,predictionDirMomentumS0,betPriceS0,stateTrackS0,meanTrackS0,meanAllS0,varAllS0,changeStateS0,truthS0,predictionS0,pTrackS0 = tradeHMM(sellData, tradeStart, nStates, 1, piVecS0, transitionMatrixS0, meanMatrixS0, varMatrixS0, npred, windowLength, 0.1, discount)
endExec = time.time()
totalTime = endExec - startExec
print "total time spent: %f" % totalTime
print "HMM Buy accuracy: ", sum(truthDirB0 == predictionDirB0)/len(truthDirB0)
print "Momentum HMM Buy accuracy: ", sum(truthDirB0 == predictionDirMomentumB0)/len(truthDirB0)
print "HMM Sell accuracy: ", sum(truthDirS0 == predictionDirS0)/len(truthDirS0)
print "Momentum HMM Sell accuracy: ", sum(truthDirS0 == predictionDirMomentumS0)/len(truthDirS0)

total time spent: 893.836000
HMM Buy accuracy:  0.363987792472
Momentum HMM Buy accuracy:  0.506408952187
HMM Sell accuracy:  0.383757205832
Momentum HMM Sell accuracy:  0.522414377755


C:\Users\draj36\Anaconda2\lib\site-packages\IPython\kernel\__main__.py:34: RuntimeWarning: divide by zero encountered in log


In [215]:
np.savetxt('truthDirBuy_0.txt', truthDirB0, delimiter=',')
np.savetxt('truthDirSell_0.txt', truthDirS0, delimiter=',')
np.savetxt('predictionDirBuyMomentum_0_2.txt', predictionDirMomentumB0, delimiter=',')
np.savetxt('predictionDirSellMomentum_0_2.txt', predictionDirMomentumS0, delimiter=',')
np.savetxt('predictionDirBuy_0_2.txt', predictionDirB0, delimiter=',')
np.savetxt('predictionDirSell_0_2.txt', predictionDirS0, delimiter=',')
np.savetxt('predictionBuy_0_2.txt', predictionB0, delimiter=',')
np.savetxt('predictionSell_0_2.txt', predictionS0, delimiter=',')

In [216]:
with file('meanBuy_0_2.txt', 'w') as outfile:
    for data_slice in meanAllB0:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('meanSell_0_2.txt', 'w') as outfile:
    for data_slice in meanAllS0:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varBuy_0_2.txt', 'w') as outfile:
    for data_slice in varAllB0:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')
with file('varSell_0_2.txt', 'w') as outfile:
    for data_slice in varAllS0:
        np.savetxt(outfile, data_slice, fmt='%-7.7f')

In [217]:
new_data = np.loadtxt('meanBuy_0_2.txt')
new_data = new_data.reshape((2,npred))
print(np.mean(abs(new_data-meanAllB0)))

6.16717422672e-09


In [257]:
print "RMSE Buy: ", np.sqrt(np.mean(abs(predictionB0-truthB)**2))
print "RMSE Sell: ", np.sqrt(np.mean(abs(predictionS0-truthS)**2))

RMSE Buy:  2.65350911392
RMSE Sell:  4.78980732505


In [269]:
profit0,a0,b0 = simple_trade(betPriceB0[1:],betPriceS0[1:],truthB0[-1],truthS0[-1],predictionDirB0[1:],predictionDirS0[1:])
print "Profit: $", profit0

Profit: $ -1058.24999999


In [270]:
plt.plot(bankTracker2,'r',label='HMM trade')
plt.plot(1000*np.ones(len(bankTracker2)),'b',label='do nothing')
plt.ylabel('Account Value')
plt.xlabel('Day')
plt.title('HMM Trading Account Value K = 2')
plt.legend(loc=(0,.9))
plt.show()

In [259]:
meanB0 = np.copy(meanAllB0)
for ii in range(0,2):
    for jj in range(0,len(meanB0[ii,:])):
        if meanB0[ii,jj] == 0:
            meanB0[ii,jj] = meanB0[ii,jj-1]

meanS0 = np.copy(meanAllS0)
for ii in range(0,2):
    for jj in range(0,len(meanS0[ii,:])):
        if meanS0[ii,jj] == 0:
            meanS0[ii,jj] = meanS0[ii,jj-1]

In [260]:
plt.plot(meanB0[0,:],'b')
plt.plot(meanB0[1,:],'r')
plt.plot(truthB0,'g')
plt.show()

In [265]:
plt.plot(meanS0[0,:],'b')
plt.plot(meanS0[1,:],'r')
plt.plot(truthS0,'g')
plt.show()

In [273]:
fig, ax = plt.subplots()
npred = len(predictionB0)
#npred = 500
xaxis = timesPred[0:npred]
ax.plot(xaxis,meanB0[0,0:npred],'b',label='Mean 1')
ax.fill_between(xaxis, meanB0[0,0:npred]-1.96*np.sqrt(varAllB0[0,0:npred]), meanB0[0,0:npred]+1.96*np.sqrt(varAllB0[0,0:npred]), alpha=0.3, color='b',label='2-sigma confidence')
ax.plot(xaxis,meanB0[1,0:npred],'r',label='Mean 2')
ax.fill_between(xaxis, meanB0[1,0:npred]-1.96*np.sqrt(varAllB0[1,0:npred]), meanB0[1,0:npred]+1.96*np.sqrt(varAllB0[1,0:npred]), alpha=0.1, color='r',label='2-sigma confidence')
ax.plot(xaxis,truthB0[0:npred],'g',label='Truth')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(xaxis,meanB0[0,0:npred],'b',label='Mean 1')
ax.fill_between(xaxis, meanB0[0,0:npred]-1.96*np.sqrt(varAllB0[0,0:npred]), meanB0[0,0:npred]+1.96*np.sqrt(varAllB0[0,0:npred]), alpha=0.3, color='b',label='2-sigma confidence')
ax.plot(xaxis,meanB0[1,0:npred],'r',label='Mean 2')
ax.fill_between(xaxis, meanB0[1,0:npred]-1.96*np.sqrt(varAllB0[1,0:npred]), meanB0[1,0:npred]+1.96*np.sqrt(varAllB0[1,0:npred]), alpha=0.1, color='r',label='2-sigma confidence')
ax.plot(xaxis,truthB0[0:npred],'g',label='Truth')
plt.legend(loc=(0,.8))
plt.ylabel("price")
plt.xlabel("time")
plt.title("BTC price and Hidden State Means with 95 percent coverage over Trading Period")
ax.set_xticklabels(labels)
plt.show()

In [189]:
fig, ax = plt.subplots()
xaxis = timesPred[0:len(predictionB0)]
ax.plot(xaxis,meanS0[0,:],'b',label='Mean 1')
ax.fill_between(xaxis, meanS0[0,:]-1.96*np.sqrt(varAllS0[0,:]), meanS0[0,:]+1.96*np.sqrt(varAllS0[0,:]), alpha=0.3, color='b',label='2-sigma confidence')
ax.plot(xaxis,meanB0[1,:],'r',label='Mean 2')
ax.fill_between(xaxis, meanS0[1,:]-1.96*np.sqrt(varAllS0[1,:]), meanS0[1,:]+1.96*np.sqrt(varAllS0[1,:]), alpha=0.1, color='r',label='2-sigma confidence')
ax.plot(xaxis,truthS0,'g',label='Truth')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(xaxis,meanS0[0,:],'b',label='Mean 1')
ax.fill_between(xaxis, meanS0[0,:]-1.96*np.sqrt(varAllS0[0,:]), meanS0[0,:]+1.96*np.sqrt(varAllS0[0,:]), alpha=0.3, color='b',label='2-sigma confidence')
ax.plot(xaxis,meanB0[1,:],'r',label='Mean 2')
ax.fill_between(xaxis, meanS0[1,:]-1.96*np.sqrt(varAllS0[1,:]), meanS0[1,:]+1.96*np.sqrt(varAllS0[1,:]), alpha=0.1, color='r',label='2-sigma confidence')
ax.plot(xaxis,truthS0,'g',label='Truth')
plt.legend(loc=(0,.8))
plt.ylabel("price")
plt.xlabel("time")
plt.title("BTC price and Hidden State Means with 95 percent coverage over Trading Period")
ax.set_xticklabels(labels)
plt.show()

In [275]:
fig, ax = plt.subplots()
xaxis = timesPred[0:len(predictionB0)]
ax.plot(xaxis,truthB0,'r',label='Bid')
ax.plot(xaxis,truthS0,'b',label='Ask')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(xaxis,truthB0,'r',label='Bid')
ax.plot(xaxis,truthS0,'b',label='Ask')
plt.legend(loc=(0,.8))
plt.ylabel("price")
plt.xlabel("time")
plt.title("OK Coin BTC Bid and Ask Period 1")
ax.set_xticklabels(labels)
plt.show()

In [190]:
fig, ax = plt.subplots()
xaxis = timesPred[0:len(predictionB)]
ax.plot(xaxis,predictionB,'b-',label='predicted price')
ax.plot(xaxis[predictionDirB==truthDirB],predictionB[predictionDirB==truthDirB],'k*',label='correct pred')
ax.plot(xaxis,truthB,'r-',label='actual price')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(xaxis,predictionB,'b-',label='predicted price')
ax.plot(xaxis[predictionDirB==truthDirB],predictionB[predictionDirB==truthDirB],'k*',label='correct pred')
ax.plot(xaxis,truthB,'r-',label='actual price')
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('BTC Price')
plt.title('HMM Prediction on OKCoin K = 2')
plt.legend(loc = 0)
plt.show()

In [227]:
def simple_trade(buyPrice,sellPrice,lastBuyPrice,lastSellPrice,predictionDirB,predictionDirS):
    bank1 = 10**3
    assets1 = 0

    bankTracker1 = [bank1]
    assetTracker1 = [0]

    betSize = .1
    
    for ii in range(0,len(buyPrice)):
    
        buyP1 = buyPrice[ii]
        sellP1 = sellPrice[ii]
        
        if predictionDirS[ii] > 0: # buy
            bank1 = bank1 - betSize*sellP1
            assets1 = assets1 + betSize
        if predictionDirB[ii] < 0: # sell
            bank1 = bank1 + betSize*buyP1
            assets1 = assets1 - betSize
            
        bankTracker1.append(bank1)
        assetTracker1.append(assets1)
        
    # close position at end
    if assets1 > 0: # sell
        bank1 = bank1 + assets1*lastBuyPrice
        assets1 = 0
    if assets1 < 0: # buy
        bank1 = bank1 + assets1*lastSellPrice
        assets1 = 0
        
    bankTracker1.append(bank1)
    assetTracker1.append(assets1)

    bankTracker1 = np.array(bankTracker1)
    assetTracker1 = np.array(assetTracker1)
    profit = bankTracker1[-1] - bankTracker1[0]
    
    return profit, bankTracker1, assetTracker1

In [354]:
np.floor(-2.5)

-3.0